In [1]:
import pandas as pd
import numpy as np

In [2]:
data_file_path='../xpto_sales_products_mar_may_2024.csv - Página4.csv'

In [3]:
data_table = pd.read_csv(data_file_path, sep=',')

In [4]:
data_table

,sale_date,product_id,product_title,product_price,product_image_url,store_name,store_id,sales_per_day
0,2024-05-05,1,Smartphone Samsung Galaxy S21,529.55,https://www.lorempixel.com/868/364,Amazon,1,8
1,2024-05-15,1,Smartphone Samsung Galaxy S21,555.87,https://www.lorempixel.com/486/36,Amazon,1,5
2,2024-04-26,1,Smartphone Samsung Galaxy S21,487.37,https://www.lorempixel.com/768/652,C&A,3,6
3,2024-05-27,1,Smartphone Samsung Galaxy S21,552.08,https://placekitten.com/937/239,Fnac,9,12
4,2024-04-23,1,Smartphone Samsung Galaxy S21,484.87,https://placekitten.com/562/482,Fnac,9,16
...,...,...,...,...,...,...,...,...
795,2024-03-08,49,Processador Intel Core i7,155.73,https://placeimg.com/96/35/any,Netshoes,15,5
796,2024-03-05,49,Processador Intel Core i7,139.52,https://dummyimage.com/46x877,Ricardo Eletro,17,18
797,2024-03-17,49,Processador Intel Core i7,91.73,https://dummyimage.com/98x145,Shoptime,19,18
798,2024-03-03,49,Processador Intel Core i7,92.79,https://dummyimage.com/151x612,Shoptime,19,4


In [5]:
d = data_table.groupby(['product_id', 'product_title'],as_index=False).agg(total_sales=('sales_per_day','sum')).sort_values('total_sales',ascending=False).head(10)
d


,product_id,product_title,total_sales
35,36,Home Theater Sony,312
46,47,HD Externo Seagate,290
28,29,Caixa de Som Bluetooth Bose,283
20,21,Ar Condicionado Split LG,262
11,12,Impressora Multifuncional HP,260
17,18,Máquina de Lavar Roupas Brastemp,242
1,2,Notebook Dell Inspiron 15,241
41,42,Termômetro Digital G-Tech,227
26,27,Drone DJI Phantom,216
2,3,Fone de Ouvido Bluetooth JBL,207


In [6]:
top_sold = list(d['product_id'])
top_sold


[36, 47, 29, 21, 12, 18, 2, 42, 27, 3]

In [18]:
top_by_store = data_table[data_table['product_id'].isin(top_sold)].groupby(['product_id', 'product_title', 'store_id', 'store_name'],as_index=False).agg(total_sales_store=('sales_per_day','sum')).sort_values('total_sales_store',ascending=False)

In [19]:
top_by_store

,product_id,product_title,store_id,store_name,total_sales_store
107,36,Home Theater Sony,20,Submarino,60
65,21,Ar Condicionado Split LG,19,Shoptime,54
56,21,Ar Condicionado Split LG,6,Centauro,48
6,2,Notebook Dell Inspiron 15,10,Havan,45
69,27,Drone DJI Phantom,3,C&A,44
...,...,...,...,...,...
30,12,Impressora Multifuncional HP,3,C&A,2
5,2,Notebook Dell Inspiron 15,9,Fnac,1
45,18,Máquina de Lavar Roupas Brastemp,6,Centauro,1
61,21,Ar Condicionado Split LG,14,Magazine Luiza,1


In [20]:
top_by_store.columns

Index(['product_id', 'product_title', 'store_id', 'store_name',
       'total_sales_store'],
      dtype='object')

In [22]:
idx = top_by_store.groupby('product_id')['total_sales_store'].idxmax()

In [23]:
idx

product_id
2       6
3      26
12     28
18     44
21     65
27     69
29     90
36    107
42    120
47    124
Name: total_sales_store, dtype: int64

In [25]:
top_by_store.loc[idx]

,product_id,product_title,store_id,store_name,total_sales_store
6,2,Notebook Dell Inspiron 15,10,Havan,45
26,3,Fone de Ouvido Bluetooth JBL,18,Saraiva,34
28,12,Impressora Multifuncional HP,1,Amazon,43
44,18,Máquina de Lavar Roupas Brastemp,5,Casas Bahia,43
65,21,Ar Condicionado Split LG,19,Shoptime,54
69,27,Drone DJI Phantom,3,C&A,44
90,29,Caixa de Som Bluetooth Bose,17,Ricardo Eletro,38
107,36,Home Theater Sony,20,Submarino,60
120,42,Termômetro Digital G-Tech,19,Shoptime,42
124,47,HD Externo Seagate,3,C&A,34


In [36]:
regs = [(d['product_id'],d['store_id']) for d in top_by_store.loc[idx][['product_id','store_id']].to_dict('records')]
regs

[(2, 10),
 (3, 18),
 (12, 1),
 (18, 5),
 (21, 19),
 (27, 3),
 (29, 17),
 (36, 20),
 (42, 19),
 (47, 3)]

In [66]:
data_table[data_table['product_id'].isin(r[0] for r in regs) & data_table['store_id'].isin(r[1] for r in regs)]
#r = regs[0]
#r
data_table[(data_table['product_id']==r[0]) & (data_table['store_id']==r[1])].to_dict('records')[0]
t = []
for r in regs:
    x = data_table[(data_table['product_id']==r[0]) & (data_table['store_id']==r[1])].to_dict('records')[0]
    print(x)
    t.append(x)


{'sale_date': '2024-04-22', 'product_id': 2, 'product_title': 'Notebook Dell Inspiron 15', 'product_price': 733.94, 'product_image_url': 'https://placekitten.com/549/41', 'store_name': 'Havan', 'store_id': 10, 'sales_per_day': 11, 'total_sales': 241, 'total_sales_store': 45}
{'sale_date': '2024-05-28', 'product_id': 3, 'product_title': 'Fone de Ouvido Bluetooth JBL', 'product_price': 569.94, 'product_image_url': 'https://placeimg.com/997/338/any', 'store_name': 'Saraiva', 'store_id': 18, 'sales_per_day': 17, 'total_sales': 207, 'total_sales_store': 34}
{'sale_date': '2024-04-14', 'product_id': 12, 'product_title': 'Impressora Multifuncional HP', 'product_price': 554.13, 'product_image_url': 'https://placekitten.com/64/831', 'store_name': 'Amazon', 'store_id': 1, 'sales_per_day': 20, 'total_sales': 260, 'total_sales_store': 43}
{'sale_date': '2024-04-05', 'product_id': 18, 'product_title': 'Máquina de Lavar Roupas Brastemp', 'product_price': 350.27, 'product_image_url': 'https://placeim

In [67]:
ads = pd.DataFrame(t)
ads

,sale_date,product_id,product_title,product_price,product_image_url,store_name,store_id,sales_per_day,total_sales,total_sales_store
0,2024-04-22,2,Notebook Dell Inspiron 15,733.94,https://placekitten.com/549/41,Havan,10,11,241,45
1,2024-05-28,3,Fone de Ouvido Bluetooth JBL,569.94,https://placeimg.com/997/338/any,Saraiva,18,17,207,34
2,2024-04-14,12,Impressora Multifuncional HP,554.13,https://placekitten.com/64/831,Amazon,1,20,260,43
3,2024-04-05,18,Máquina de Lavar Roupas Brastemp,350.27,https://placeimg.com/659/111/any,Casas Bahia,5,2,242,43
4,2024-05-03,21,Ar Condicionado Split LG,1201.25,https://placeimg.com/993/661/any,Shoptime,19,11,262,54
5,2024-04-01,27,Drone DJI Phantom,165.11,https://www.lorempixel.com/603/888,C&A,3,12,216,44
6,2024-05-27,29,Caixa de Som Bluetooth Bose,770.78,https://www.lorempixel.com/264/886,Ricardo Eletro,17,15,283,38
7,2024-04-16,36,Home Theater Sony,12.87,https://dummyimage.com/195x115,Submarino,20,5,312,60
8,2024-03-25,42,Termômetro Digital G-Tech,143.93,https://placeimg.com/775/948/any,Shoptime,19,19,227,42
9,2024-03-12,47,HD Externo Seagate,842.42,https://placeimg.com/379/103/any,C&A,3,16,290,34
